In [46]:
import tensorflow as tf
import torch as torch
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("~/PycharmProjects/Diabetes/final.csv", sep=',')

Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data[Albumin_Creatinine_data['Albumin/Creatinine_Res'] < 300]
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))

data = grouped_lists[['Albumin/Creatinine_Res']]

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 8 :
        seq = data['Albumin/Creatinine_Res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
print(y)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)

for index, row in data.iterrows() :
# demonstrate prediction
        x_input = np.asarray(seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat)
        y_actual = np.asarray(seq[-1:])
        rms = sqrt(mean_squared_error(y_actual, yhat))
        print(rms)